In [2]:
%pip install bayesian-optimization

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# implement a dummy Bayesian optimization algorithm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
from sklearn.gaussian_process.kernels import RBF, Matern # you can try to import other kernels from sklearn as well

### Prepare target flow curve

In [38]:
# Read the CSV file into a DataFrame
df = pd.read_csv('abaqus simulation F-D fitting/Flowcurve_RT.csv')
# print(df)
# Extract the true strain and average true stress columns
trueStrain = df['True strain']
trueStress = df['Avg.True stress']

In [39]:
# Continuous searching space
param_bounds = {
    "c1": (500, 2000),  
    "c2": (0.1, 10),    
    "c3": (0.01, 0.1)  
}

# Swift Laws
def swift_law(c1,c2,c3,strain):
    stress=c1*(c2+strain)**c3 #3 parameter 
    return(stress)

# Note: BO in Bayes-Opt tries to maximize, so you should use the inverse of the loss function.
def lossFunction(**solution):
    c1 = solution["c1"]
    c2 = solution["c2"]
    c3 = solution["c3"]
    simStress = swift_law(c1,c2,c3,trueStrain)
    # RMSE loss
    fitness = np.sqrt(np.mean((simStress - trueStress)**2))
    loss = 1.0 / fitness
    return loss

In [45]:
class BO():
    
    ##################################
    # OPTIMIZER CLASS INITIALIZATION #
    ##################################

    def __init__(self):        
        #############################
        # Optimizer hyperparameters #
        #############################
        
        # maximize parameters
        self.verbose = 1 # 0 for no output, 1 for some output printing
        self.random_state = 123 # random seed
        self.init_points = 100 # number of initial points to sample randomly for Bayesian optimization
        self.iterations = 100 # number of iterations to run Bayesian optimization
        
        # Acquisition function        
        # Low kappa means more exploitation for UCB
        # High kappa means more exploration for UCB
        # Low xi means more exploitation for EI and POI
        # High xi means more exploration for EI and POI
        self.acquisitionFunction = UtilityFunction(kind='ucb', kappa=2.576, xi=0, kappa_decay=1, kappa_decay_delay=0)
        #self.acquisitionFunction = UtilityFunction(kind='poi', kappa=2.576, xi=0, kappa_decay=1, kappa_decay_delay=0)
        #self.acquisitionFunction = UtilityFunction(kind='ei', kappa=2.576, xi=0, kappa_decay=1, kappa_decay_delay=0)
        
        # Gaussian process kernel parameters
        self.GP_kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-3, 1e3)) # RBF kernel
        #self.GP_kernel = Matern(nu=2.5) # Matern kernel
        self.alpha = 1e-6
        self.normalize_y=True
        self.n_restarts_optimizer=5
        
    ##########################
    # OPTIMIZATION FUNCTIONS #
    ##########################

    def initializeOptimizer(self, lossFunction, param_bounds):
        self.param_bounds = param_bounds
        BO_bounds = param_bounds
        bo_instance = BayesianOptimization(
            f = lossFunction,
            pbounds = BO_bounds, 
            verbose = self.verbose,
            random_state = self.random_state,
            bounds_transformer = None,
            allow_duplicate_points = False
        )
        bo_instance.set_gp_params(
            kernel=self.GP_kernel,
            alpha=self.alpha,
            normalize_y=self.normalize_y,
            n_restarts_optimizer=self.n_restarts_optimizer,
            random_state=self.random_state,
        )
        self.optimizer = bo_instance

    def run(self):
        self.optimizer.maximize(
            init_points = self.init_points, 
            n_iter = self.iterations,   
            acquisition_function=self.acquisitionFunction, 
        )
        
    def outputResult(self):
        solution_dict = self.optimizer.max["params"]
        solution_tuple = tuple(solution_dict.items())
        best_solution_loss = self.optimizer.max["target"]
        return solution_dict, solution_tuple, best_solution_loss

In [46]:
BO_instance = BO()
BO_instance.initializeOptimizer(lossFunction, param_bounds)
BO_instance.run()
solution_dict, solution_tuple, best_solution_loss = BO_instance.outputResult()
print(solution_dict)

for param in solution_dict:
    print(f"{param}: {solution_dict[param]}")

print(f"Best solution loss = {best_solution_loss:.4f}")

|   iter    |  target   |    c1     |    c2     |    c3     |
-------------------------------------------------------------
| 2         | 0.00201   | 1.327e+03 | 7.223     | 0.04808   |
| 4         | 0.004105  | 1.088e+03 | 3.497     | 0.07561   |
| 5         | 0.005213  | 1.158e+03 | 0.6908    | 0.04582   |
| 10        | 0.009144  | 842.4     | 3.008     | 0.06679   |
| 16        | 0.009564  | 875.7     | 4.882     | 0.0987    |
| 43        | 0.009605  | 886.3     | 5.687     | 0.08263   |
| 52        | 0.0107    | 959.7     | 6.686     | 0.02003   |
| 77        | 0.01107   | 970.5     | 0.5687    | 0.03175   |
| 103       | 0.01114   | 916.4     | 9.914     | 0.02738   |
| 125       | 0.01121   | 957.5     | 1.024     | 0.07529   |
| 148       | 0.01122   | 938.7     | 1.433     | 0.07365   |
| 166       | 0.01134   | 1.02e+03  | 0.1164    | 0.03435   |
{'c1': 1020.354389809241, 'c2': 0.11635268154942591, 'c3': 0.03434589503698687}
c1: 1020.354389809241
c2: 0.11635268154942591
c3: 0.

In [ ]:
# Saving, loading and restarting